# polymake file format and the polyDB

## Polymake Data Files

### JSON File Format

With release 4.0 polymake adopts a new file format for its data files. 
In the new version all data is now serialized to JSON instead of XML.

polymake can still read files in XML format.

In [1]:
$p = new Polytope(POINTS=>[[1,0,0],[1,1,0],[1,0,1]]);

In [2]:
print JSON->new->pretty->encode(Core::Serializer::serialize($p));

{
  "_ns" : {
    "polymake" : [
      "https://polymake.org",
      "4.0"
    ]
  },
  "POINTS" : [
    [
      "1",
      "0",
      "0"
    ],
    [
      "1",
      "1",
      "0"
    ],
    [
      "1",
      "0",
      "1"
    ]
  ],
  "_id" : "p",
  "CONE_AMBIENT_DIM" : 3,
  "_type" : "polytope::Polytope<Rational>"
}


JSON is a pretty simple format. Entries in a JSON object are either
 * number: no distinction between float and int
 * string
 * boolean: true/false
 * array: comma separated list of elements in square brackets
 * object: comma separeted key-value pairs in curly brackets. associative array
 * null

Polymake stores 
 * perl builtin numbers a number
 * vectors, matrices as array
 * tuples as object
 * Integer/Rartional as string
 * Bool as boolean
 * big objects as objects (with properties as keys)
  

Type information that cannot be infered from the rule base (sparse, attachment) is stored in an separate key-value pair.

There are only a few meta tags left in a data file:
 * _id: name
 * _ns: mark it as a polymake file
 * _info: credits and description
 * _type: big object type
 * _attrs: additional attributes for properties, marks e.g. attachments, non-standard types (e.g. sparse)
 * _ext: extensions
 * _load: further applications needed to read data file
 * _polyDB: database information, if object is retrieved from db

### Object Arrays

You can now use standard perl arrays to store a list of objects.

In [3]:
$cube = cube(2);
$simplex = simplex(2);
$array = [ $cube, $simplex ];

In [4]:
print JSON->new->pretty->encode(Core::Serializer::serialize($array));

{
  "data" : [
    {
      "_type" : "polytope::Polytope<Rational>",
      "_info" : {
        "description" : "cube of dimension 2\n"
      },
      "FACETS" : [
        {
          "0" : "1",
          "1" : "1"
        },
        {
          "0" : "1",
          "1" : "-1"
        },
        {
          "0" : "1",
          "2" : "1"
        },
        {
          "0" : "1",
          "2" : "-1"
        },
        {
          "cols" : 3
        }
      ],
      "VERTICES_IN_FACETS" : [
        [
          0,
          2
        ],
        [
          1,
          3
        ],
        [
          0,
          1
        ],
        [
          2,
          3
        ],
        {
          "cols" : 4
        }
      ],
      "_attrs" : {
        "FACETS" : {
          "_type" : "SparseMatrix<Rational, NonSymmetric>"
        }
      },
      "BOUNDED" : true,
      "CONE_AMBIENT_DIM" : 3,
      "CONE_DIM" : 3,
      "AFFINE_HULL" : [
        {
          "cols" : 3
        }
      ],
    

### Background: JSON Schemata

In [5]:
$p = new Polytope(VERTICES=>[[1,0,0],[1,1,0],[1,0,1]]);
$p->N_VERTICES;

In [6]:
$schema = create_restrictive_schema($p);

In [7]:
print JSON->new->pretty->encode($schema->source);

{
  "definitions" : {
    "common-Matrix-Rational-NonSymmetric" : {
      "type" : "array",
      "items" : {
        "oneOf" : [
          {
            "$ref" : "#/definitions/common-Vector-Rational"
          },
          {
            "properties" : {
              "cols" : {
                "type" : "integer",
                "minimum" : 0
              }
            },
            "required" : [
              "cols"
            ],
            "type" : "object",
            "additionalProperties" : false
          }
        ]
      }
    },
    "common-Rational" : {
      "type" : "string",
      "pattern" : "^-?(\\d+(/\\d+)?|inf)$"
    },
    "common-Vector-Rational" : {
      "type" : "array",
      "items" : {
        "$ref" : "#/definitions/common-Rational"
      }
    },
    "common-Int" : {
      "type" : "integer"
    }
  },
  "additionalProperties" : false,
  "required" : [
    "_ns",
    "_type",
    "VERTICES",
    "CONE_AMBIENT_DIM",
    "N_VERTICES"
  ],
  "properties"

In [8]:
print $schema->validate(Core::Serializer::serialize($p));

true

We can use a schema to modify a data file so that it validates against the schema

In [9]:
$q = new Polytope(VERTICES=>[[1,0,0],[1,1,0],[1,0,1],[1,1,1]]);

In [10]:
print JSON->new->pretty->encode(Core::Serializer::serialize($q));

{
  "_type" : "polytope::Polytope<Rational>",
  "VERTICES" : [
    [
      "1",
      "0",
      "0"
    ],
    [
      "1",
      "1",
      "0"
    ],
    [
      "1",
      "0",
      "1"
    ],
    [
      "1",
      "1",
      "1"
    ]
  ],
  "CONE_AMBIENT_DIM" : 3,
  "_ns" : {
    "polymake" : [
      "https://polymake.org",
      "4.0"
    ]
  },
  "_id" : "q"
}


In [11]:
print JSON->new->pretty->encode(Core::Serializer::serialize($q, { schema => $schema } ));

{
  "VERTICES" : [
    [
      "1",
      "0",
      "0"
    ],
    [
      "1",
      "1",
      "0"
    ],
    [
      "1",
      "0",
      "1"
    ],
    [
      "1",
      "1",
      "1"
    ]
  ],
  "_type" : "polytope::Polytope<Rational>",
  "N_VERTICES" : 4,
  "CONE_AMBIENT_DIM" : 3,
  "_ns" : {
    "polymake" : [
      "https://polymake.org",
      "4.0"
    ]
  },
  "_id" : "q"
}


In [12]:
help "prescribe_property_types";

core/objects/Schema/methods/prescribe_property_types:
prescribe_property_types(properties;  ... )

 Change the stored data type for properties in a restrictive schema

 Selected properties will be coerced to a different type when the object is stored in a data file
 or in PolyDB using this schema.

 Note: This function does not try to verify that the properties can be serialized as the given type
 or will be successfully deserialized when reading the object from a file or retrieving it from a database.
 It's solely in the users responsibility to choose a compatible and convertible data type.

Arguments:
  Any properties list of pairs PROPERTY_NAME => type
    A property in a subobject is written in dotted path notation: "NAME1.NAME2.NAME3..."
    Types can be specified by name in a string or as a typeof expression or as a type of an exsiting data object.
    Specifying undef or "default" will enforce storing the properties in their original form, as declared in the rulebase.
    This o

We can change the type of VERTICES in our schema:

In [13]:
$schema->prescribe_property_types(VERTICES => typeof SparseMatrix<Rational>);

In [14]:
print JSON->new->pretty->encode(Core::Serializer::serialize($q, { schema => $schema } ));

{
  "VERTICES" : [
    {
      "0" : "1"
    },
    {
      "0" : "1",
      "1" : "1"
    },
    {
      "0" : "1",
      "2" : "1"
    },
    [
      "1",
      "1",
      "1"
    ],
    {
      "cols" : 3
    }
  ],
  "_type" : "polytope::Polytope<Rational>",
  "_attrs" : {
    "VERTICES" : {
      "_type" : "SparseMatrix<Rational, NonSymmetric>"
    }
  },
  "N_VERTICES" : 4,
  "CONE_AMBIENT_DIM" : 3,
  "_id" : "q",
  "_ns" : {
    "polymake" : [
      "https://polymake.org",
      "4.0"
    ]
  }
}


We can also create a schema for a complete big object type

In [15]:
$full_schema=create_permissive_schema(typeof graph::Graph);

In [16]:
print JSON->new->pretty->encode($full_schema->source);

{
  "$schema" : "http://json-schema.org/draft-07/schema#",
  "definitions" : {
    "common-Array-Int" : {
      "type" : "array",
      "items" : {
        "$ref" : "#/definitions/common-Int"
      }
    },
    "common-SparseVector-Int" : {
      "type" : [
        "array",
        "object"
      ],
      "items" : {
        "$ref" : "#/definitions/common-Int"
      },
      "additionalProperties" : false,
      "patternProperties" : {
        "^\\d+$" : {
          "$ref" : "#/definitions/common-Int"
        }
      },
      "properties" : {
        "_dim" : {
          "minimum" : 0,
          "type" : "integer"
        }
      }
    },
    "common-Int" : {
      "type" : "integer"
    },
    "common-SparseVector-Float" : {
      "patternProperties" : {
        "^\\d+$" : {
          "$ref" : "#/definitions/common-Float"
        }
      },
      "additionalProperties" : false,
      "properties" : {
        "_dim" : {
          "minimum" : 0,
          "type" : "integer"
        }
  

In [17]:
$m=graph::petersen();

In [18]:
print $full_schema->validate(Core::Serializer::serialize($m));

true

## PolyDB

### General Information

Database for mathematical objects. Intended for data related to the projects within OSCAR. 

Database is independent of particular software package, but information necessary for in-/output to a software can be stored alongside.

Currently accessible via web (still at an experimental stage) and via polymake.

https://polydb.org

A database connection is established by calling the function ```polyDB```:

In [20]:
$pdb_default=polyDB();

Click here for additional output 
 
polymake: used package polyDB
 Access to the polymake polytope database.
 (c) 2013-2019 Silke Horn, Andreas Paffenholz
 http://www.mathematik.tu-darmstadt.de/~paffenholz

### Get info

In [21]:
$pdb_default->info();

available polydb collections

SECTION: Manifolds
This database contains combinatorial manifolds

COLLECTION: DIM2_3
This is a collection of  combinatorial 2-  and  3-manifolds with up to 10 vertices given as triangulations and calculcated by Frank Lutz found at: http://page.math.tu-berlin.de/~lutz/stellar/mixed.html.

SECTION: Matroids
This database contains various classes of matroids.

COLLECTION: Small
Collection of small matroids based on the census by Yoshitake Matsumoto, Sonoko Moriyama, Hiroshi Imai, David Bremner.

SECTION: Polytopes
A collection of families of polytopes

SECTION: Polytopes.Combinatorial
This database contains various classes of combinatorial polytopes.

COLLECTION: 01Polytopes
0/1-Polytopes up to combinatorial equivalence

COLLECTION: CombinatorialTypes
Combinatorial types of polytopes of small dimension and number of vertices

COLLECTION: FacesBirkhoffPolytope
Combinatorial types of faces up to dimension  8 of a Birkhoff polytope in any dimension

COLLECTION:

For individual collections we can get more detailed information:
 * level 1: short description (default if no collection is given),
 * level  2: description, 
 * level 3: description, authors, maintainers, 
 * level 4: full info
 * level 5: full info and list of recommended search fields, if defined in info (default if collection is given)

In [22]:
$pdb_default->info(section=>"Polytopes.Combinatorial", collection=>"CombinatorialTypes", info_level=>4);

available polydb collections

SECTION: Polytopes.Combinatorial
This database contains various classes of combinatorial polytopes.

COLLECTION: CombinatorialTypes
Combinatorial types of polytopes of small dimension and number of vertices, see [[http://www-imai.is.s.u-tokyo.ac.jp/~hmiyata/oriented_matroids/index.html]]
Author(s): 
  Hiroyuki Miyata, University of Tokyo, Bunkyo-ku, Tokyo, Japan, hmiyata@is.s.u-tokyo.ac.jp
  Sonoko Moriyama, Tokuyama laboratory, System Information Sciences, Graduate School of Information Sciences, Tohoku University, moriso@dais.is.tohoku.ac.jp 
  Komei Fukuda, Theory of Combinatorial Algorithms, Department of Mathematics, ETHZ, fukuda@math.ethz.ch
Contributor(s): 
  Constantin Fischer, TU Berlin, cfischer@mailbox.tu-berlin.de
Maintainer(s): 
  Constantin Fischer, TU Berlin, cfischer@mailbox.tu-berlin.de
Online Resources:
Classification of Oriented Matroids: http://www-imai.is.s.u-tokyo.ac.jp/~hmiyata/oriented_matroids/index.html
Searchable fields in this c

### Reading Data from polymake

#### Simple reading

In [23]:
$coll_default=$pdb_default->get_collection("Polytopes.Lattice.SmoothReflexive");

In [24]:
$a=$coll_default->find({"DIM"=>3});

This returns a cursor over the data.

In [25]:
print $a->has_next;

true

In [26]:
$p=$a->next;

In [27]:
print $p->F_VECTOR;

8 12 6

In [28]:
while ( $p = $a->next ) {
  print $p->F_VECTOR, "\n";
}

6 9 5
10 15 7
12 18 8
10 15 7
8 12 6
8 12 6
10 15 7
10 15 7
12 18 8
8 12 6
8 12 6
8 12 6
6 9 5
6 9 5
8 12 6
6 9 5
4 6 4


#### Objects from the database know where they come from

In [29]:
$a=$coll_default->find({"DIM"=>3});
$p=$a->next;

In [30]:
$info=$p->get_attachment("_polyDB");
foreach (keys %$info) { print $_, ": ", $info->{$_}, "\n"; };

app: polytope
section: Polytopes.Lattice
version: 2.1
uri: http://polymake.org/polytopes/paffenholz/www/fano.html
id: F.3D.0000
database: LatticePolytopes
tag: object
collection: SmoothReflexive
creation_date: 2019-08-01
type: polytope::Polytope<Rational>


### Queries

In [31]:
$cur=$coll_default->find({"DIM"=>3, "N_VERTICES" => 8});

A query usually returns a cursor over all matching results.

In [32]:
while ( $p = $cur->next ) {
  print "(", $p->DIM, ", ", $p->N_VERTICES, ") ";
}

(3, 8) (3, 8) (3, 8) (3, 8) (3, 8) (3, 8) (3, 8) 

The cursor keeps its position, so to return to the first result we need to reset the cursor.

In [33]:
$cur->reset;

In [34]:
while ( $p = $cur->next ) {
  print "(", $p->DIM, ", ", $p->N_VERTICES, ") ";
}

(3, 8) (3, 8) (3, 8) (3, 8) (3, 8) (3, 8) (3, 8) 

We can collect all results into an array.

In [35]:
@a=$coll_default->find({"DIM"=>3, "N_VERTICES" => 8})->all;

In [36]:
print scalar(@a);

7

We can ask for ranges:

In [37]:
@a=$coll_default->find({"DIM" => { '$lt' => 4}})->all;

In [38]:
print scalar(@a);

25

In [39]:
foreach (@a) {
  print $_->F_VECTOR, "\n";
}


2
3 3
4 4
4 4
5 5
6 6
8 12 6
6 9 5
10 15 7
12 18 8
10 15 7
8 12 6
8 12 6
10 15 7
10 15 7
12 18 8
8 12 6
8 12 6
8 12 6
6 9 5
6 9 5
8 12 6
6 9 5
4 6 4


Boolean operations are possible:

In [40]:
@a=$coll_default->find({'$or' => [{ 'DIM' => 2}, { 'DIM' => 4}]})->all;

In [41]:
foreach (@a) {
  print $_->F_VECTOR, "\n";
}

3 3
4 4
4 4
5 5
6 6
11 22 18 7
8 16 14 6
17 34 25 8
13 26 20 7
15 30 23 8
18 36 27 9
15 30 23 8
16 32 24 8
12 24 19 7
12 24 19 7
15 30 23 8
16 32 24 8
16 32 24 8
12 24 19 7
16 32 24 8
16 32 24 8
16 32 24 8
13 26 20 7
12 24 19 7
12 24 19 7
9 18 15 6
21 42 30 9
17 34 25 8
17 34 25 8
23 46 32 9
30 60 40 10
13 26 20 7
17 34 25 8
15 30 23 8
18 36 27 9
16 32 24 8
15 30 23 8
12 24 19 7
17 34 25 8
21 42 30 9
21 42 30 9
18 36 26 8
24 48 33 9
18 36 26 8
17 34 25 8
16 32 24 8
13 26 20 7
13 26 20 7
12 24 19 7
15 30 23 8
12 24 19 7
9 18 15 6
20 40 29 9
24 48 34 10
20 40 29 9
16 32 24 8
16 32 24 8
20 40 29 9
20 40 29 9
24 48 34 10
20 40 29 9
24 48 34 10
20 40 29 9
20 40 29 9
16 32 24 8
20 40 29 9
24 48 34 10
20 40 29 9
21 42 30 9
20 40 29 9
24 48 34 10
24 48 34 10
20 40 29 9
20 40 29 9
16 32 24 8
16 32 24 8
16 32 24 8
16 32 24 8
12 24 19 7
25 50 35 10
30 60 41 11
36 72 48 12
20 40 29 9
24 48 34 10
20 40 29 9
16 32 24 8
16 32 24 8
16 32 24 8
20 40 29 9
16 32 24 8
12 24 19 7
17 34 25 8
16 32 24 8
16 3

In [42]:
@a=$coll_default->find({'$and' => [{ 'DIM' => 2}, { 'N_VERTICES' => 4}]})->all;

In [43]:
foreach (@a) {
  print $_->F_VECTOR, "\n";
}

4 4
4 4


Type matters

In [44]:
@a=$coll_default->find({'LATTICE_VOLUME' => "8"})->all;

In [45]:
print scalar(@a);

2

In [46]:
@a=$coll_default->find({'LATTICE_VOLUME' => 8})->all;

In [47]:
print scalar(@a);

0

### Creating and filling new collections

Some preparations for a test database

In [49]:
$PolyDB::default::db_host="db-test-server";
$PolyDB::default::useSSL = 0;
$pdb_local_admin = polyDB(user=>"admin", password=>"admin");
$pdb_local_admin->create_default_user_and_role();

We initiate a new collection. MongoDB is lazy, so what this function actually does is just creating access permisssions. The collection is only created when the first document is inserted.

In [50]:
$pdb_local_admin->initiate_collection(collection=>"Polytopes.01Polytopes");

We insert an info document for the section. This can only be done by database admins. A documentation entry is necessary, otherwise the section is not listed by the ```info()```-function.

In [51]:
$section_doc=load_json("files/json_and_polydb/polydb_meta/Polytopes/Polytopes.2.1.json");
print JSON->new->pretty->encode($section_doc);

{
  "description" : "A collection of families of polytopes",
  "sectionDepth" : 1,
  "section" : [
    "Polytopes"
  ],
  "maintainer" : {
    "email" : "paffenholz@opt.tu-darmstadt.de",
    "affiliation" : "TU Darmstadt",
    "name" : "Andreas Paffenholz"
  },
  "short_description" : "A collection of families of polytopes",
  "polydb_version" : "2.1"
}


In [52]:
$pdb_local_admin->set_section_doc($section_doc, section=>"Polytopes");

From now on we can hand over the administration of the collection to a user that has admin permissions just on this collection, not the whole database. We create a new one, we could also extend an existing user.

In [53]:
$pdb_local_admin->create_user(user=>"workshop", password=>"workshop", admin_collections=>["Polytopes.01Polytopes"]);

In [54]:
print $pdb_local_admin->user_exists("workshop");

true

Now we can connect as the newly created user with admin rights on the new collection.

In [55]:
$pdb_local = polyDB(user=>"workshop", password=>"workshop");
$coll_local = $pdb_local->get_collection("Polytopes.01Polytopes");

We first set the documentation for the collection. Again, without a documentation entry the collection is not listed by the ```info()``` command.

In [56]:
$collection_doc=load_json("files/json_and_polydb/polydb_meta/Polytopes/01Polytopes/Polytopes.01Polytopes.2.1.json");
print JSON->new->pretty->encode($collection_doc);

{
  "contributor" : [
    {
      "affiliation" : "TU Darmstadt",
      "email" : "paffenholz@opt.tu-darmstadt.de",
      "name" : "Andreas Paffenholz"
    }
  ],
  "description" : "0/1-Polytopes up to rotation and reflection",
  "collection" : "01Polytopes",
  "polydb_version" : "2.1",
  "_id" : "Polytopes.01Polytopes.2.1",
  "maintainer" : [
    {
      "name" : "Andreas Paffenholz",
      "email" : "paffenholz@opt.tu-darmstadt.de",
      "affiliation" : "TU Darmstadt"
    }
  ],
  "section" : [
    "Polytopes"
  ],
  "author" : [
    {
      "name" : "Oswin Aichholzer"
    }
  ],
  "short_description" : "0/1-Polytopes up to rotation and reflection"
}


In [57]:
$coll_local->set_collection_doc($collection_doc);

We need to insert two further documents before we can insert data:
 * a schema
 * an info document with some meta information on the collection.
The second needs the id of the schema, so the schema must be inserted first.

We create a schema from a polymake object.

In [58]:
$a = load_data("files/json_and_polydb/data/geometric_01.pdata");

In [59]:
print join " ", $a->[0]->list_properties;

CONE_AMBIENT_DIM VERTICES

In [60]:
$p=$a->[0];
$p->provide("VERTICES","FACETS","F_VECTOR","SIMPLE","SIMPLICIAL");

Click here for additional output 
 
polymake: used package ppl
 The Parma Polyhedra Library ([[wiki:external_software#PPL]]): A C++ library for convex polyhedra
 and other numerical abstractions.
 http://www.cs.unipr.it/ppl/

In [61]:
print join " ", $p->list_properties;

CONE_AMBIENT_DIM VERTICES FEASIBLE N_VERTICES POINTED BOUNDED FAR_FACE AFFINE_HULL CONE_DIM LINEALITY_DIM LINEALITY_SPACE COMBINATORIAL_DIM SIMPLICITY SIMPLICIALITY SIMPLE SIMPLICIAL FACETS N_FACETS F_VECTOR

In [62]:
$schema=create_restrictive_schema($p);
print JSON->new->pretty->encode($schema->source);

{
  "definitions" : {
    "common-Set-Int" : {
      "type" : "array",
      "uniqueItems" : true,
      "items" : {
        "$ref" : "#/definitions/common-Int"
      }
    },
    "common-Int" : {
      "type" : "integer"
    },
    "common-Vector-Rational" : {
      "items" : {
        "$ref" : "#/definitions/common-Rational"
      },
      "type" : "array"
    },
    "common-Integer" : {
      "type" : "string",
      "pattern" : "^-?(\\d+|inf)$"
    },
    "common-Bool" : {
      "type" : "boolean"
    },
    "common-Vector-Integer" : {
      "items" : {
        "$ref" : "#/definitions/common-Integer"
      },
      "type" : "array"
    },
    "common-Rational" : {
      "pattern" : "^-?(\\d+(/\\d+)?|inf)$",
      "type" : "string"
    },
    "common-Matrix-Rational-NonSymmetric" : {
      "items" : {
        "oneOf" : [
          {
            "$ref" : "#/definitions/common-Vector-Rational"
          },
          {
            "additionalProperties" : false,
            "required" 

In [63]:
$coll_local->set_schema($schema->source);

Now we can insert the info document.

In [64]:
$info_doc=load_json("files/json_and_polydb/polydb_meta/Polytopes/01Polytopes/info.2.1.json");
print JSON->new->pretty->encode($info_doc);

{
  "_id" : "info.2.1",
  "polydb_version" : "2.1",
  "packages" : {
    "polymake" : {
      "type" : "polytope::Polytope<Rational>",
      "version" : "3.4"
    }
  },
  "description" : "0/1 Polytopes",
  "collection" : "01Polytopes",
  "contributor" : "Andreas Paffenholz",
  "creator" : "Oswin Aichholzer",
  "uri" : "https://polymake.org",
  "section" : [
    "Polytopes"
  ],
  "maintainer" : "Andreas Paffenholz"
}


In [65]:
$coll_local->set_info($info_doc, schema_id=>"schema.2.1");

Now the new collection is fully prepared and is listed with ```info()```:

In [66]:
$pdb_local->info();

available polydb collections

SECTION: Polytopes
A collection of families of polytopes

COLLECTION: 01Polytopes
0/1-Polytopes up to rotation and reflection


We can now insert our data.

In [67]:
$coll_local->insert(@$a);

In [68]:
print $coll_local->count({});

14

In [69]:
@b=$coll_local->find({})->all;

In [70]:
print scalar(@b);

14

In [71]:
$p=$coll_local->find_one({});

In [72]:
print join " ", $p->list_properties;

FACETS LINEALITY_DIM BOUNDED AFFINE_HULL CONE_AMBIENT_DIM N_FACETS COMBINATORIAL_DIM SIMPLICITY VERTICES SIMPLICIALITY SIMPLE FAR_FACE FEASIBLE POINTED SIMPLICIAL LINEALITY_SPACE F_VECTOR CONE_DIM N_VERTICES

In [73]:
$coll_local->remove($p->name);

In [74]:
print $coll_local->count({});

13